In [45]:
%load_ext autoreload
%autoreload 2

import json
import pprint

from pgoapi import PGoApi
from pgoapi.utilities import f2i
from stepper import Stepper, _get_cellid
from geopy.geocoders import GoogleV3
from collections import defaultdict

config = json.load(open('config.json'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import random 
METER_TO_DEG = 1. / (60 * 1852)

def fuzz(pos, tolerance=10):
    lat, lon, alt = pos
    n0 = random.normalvariate(0, tolerance * METER_TO_DEG) 
    n1 = random.normalvariate(0, tolerance * METER_TO_DEG)
    return lat + n0, lon + n1, alt

In [26]:
geolocator = GoogleV3(api_key=config['gmapkey'])
loc = geolocator.geocode(config['location'])
pos = loc.latitude, loc.longitude, loc.altitude

In [27]:
api = PGoApi()
api.set_position(*fuzz(pos))
api.login(config['auth_service'], config['username'], config['password'])
api.get_player()
api.call()

{'auth_ticket': {'end': 'Vcof1JC+qZGSkkOkLFwlpQ==\n',
  'expire_timestamp_ms': 1469255321401L,
  'start': 'Z8onlZqHanh6u/sCyUO87gitM0RlKwydIpyQVHkQsEjNoX5DLdJfhFcQJ8tpn3DtMK9B6lsuAlEz\nmP4iRh20tA==\n'},
 'direction': 1,
 'responses': {'GET_PLAYER': {'profile': {'avatar': {},
    'creation_time': 1469076398123L,
    'currency': [{'type': u'POKECOIN'},
     {'amount': 20000, 'type': u'STARDUST'}],
    'daily_bonus': {},
    'item_storage': 350,
    'poke_storage': 250,
    'tutorial': 'AAEDBAc=\n',
    'unknown12': '',
    'unknown13': '',
    'username': u'Seer1025'},
   'unknown1': 1}},
 'unknown2': 8145806132888207460L}

In [ ]:
timestamp = "\000" * 21
cellid = _get_cellid(*pos[:2])
api.get_map_objects(latitude=f2i(pos[0]),
                    longitude=f2i(pos[1]),
                    since_timestamp_ms=timestamp,
                    cell_id=cellid)
api.call()

In [28]:
api.get_inventory()
res = api.call()

In [29]:
pokemons = [item['inventory_item_data']['pokemon']
            for item in res['responses']['GET_INVENTORY']['inventory_delta']['inventory_items']
            if 'pokemon'    in item['inventory_item_data'] and
               'is_egg' not in item['inventory_item_data']['pokemon']]


In [47]:
k = 2

transfer_ids = []
transfer_infos = []

groups = defaultdict(list)
for obj in pokemons:
    groups[obj['pokemon_id']].append(obj)

for id, group in groups.items():
    if len(group) > k:
        for obj in sorted(group, key=lambda x: x['cp'], reverse=True)[k:]:
            transfer_ids.append(obj['id'])
            transfer_infos.append((id, obj['cp']))

In [50]:
transfer_infos

[(133, 107),
 (133, 107),
 (16, 177),
 (16, 177),
 (21, 147),
 (21, 120),
 (41, 182),
 (41, 129),
 (46, 201),
 (46, 163),
 (48, 307),
 (48, 245),
 (54, 193),
 (54, 187),
 (72, 157),
 (72, 111),
 (77, 135),
 (77, 58),
 (92, 127),
 (92, 90),
 (102, 140),
 (102, 105),
 (19, 121),
 (19, 121)]

In [52]:
sorted([(p['pokemon_id'], p['cp']) for p in pokemons])

[(1, 95),
 (7, 186),
 (10, 51),
 (13, 58),
 (13, 68),
 (14, 78),
 (16, 42),
 (16, 58),
 (16, 59),
 (16, 59),
 (16, 91),
 (16, 99),
 (16, 112),
 (16, 113),
 (16, 117),
 (16, 119),
 (16, 124),
 (16, 142),
 (16, 161),
 (16, 177),
 (16, 177),
 (19, 50),
 (19, 51),
 (19, 52),
 (19, 52),
 (19, 53),
 (19, 54),
 (19, 62),
 (19, 64),
 (19, 66),
 (19, 67),
 (19, 69),
 (19, 71),
 (19, 121),
 (19, 121),
 (20, 57),
 (21, 60),
 (21, 77),
 (21, 100),
 (21, 120),
 (21, 147),
 (23, 52),
 (29, 150),
 (41, 59),
 (41, 59),
 (41, 61),
 (41, 68),
 (41, 87),
 (41, 111),
 (41, 116),
 (41, 121),
 (41, 127),
 (41, 129),
 (41, 182),
 (42, 81),
 (43, 169),
 (46, 142),
 (46, 163),
 (46, 201),
 (48, 160),
 (48, 245),
 (48, 307),
 (52, 73),
 (54, 69),
 (54, 74),
 (54, 100),
 (54, 127),
 (54, 154),
 (54, 158),
 (54, 187),
 (54, 193),
 (55, 162),
 (58, 321),
 (69, 219),
 (72, 55),
 (72, 55),
 (72, 111),
 (72, 157),
 (77, 57),
 (77, 58),
 (77, 135),
 (79, 106),
 (79, 144),
 (84, 120),
 (92, 86),
 (92, 90),
 (92, 127),
